# Discriminative Deep Learning Imputation Method Trials

Most of the notebook is similar to the hotDeck trials notebook save for the final part where the model is built and tested.

This notebook's creating was neccesiated by the different data structure of the predictions from the deep learning imputer.

In [1]:
%cd ..

c:\Users\nick\OneDrive\Desktop\Prospect 33\Mini_DIVA


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/My Drive/Mini_DIVA'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import losses, metrics

from imputers.discriminativeImputer import discriminativeDLImputer

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

## Loading data

In [3]:
# read the data
file_dir = "../Mini_DIVA/datasets/Automobile.csv"

df = pd.read_csv(file_dir)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
1,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
2,1,158,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.4,8.5,110,5500,19,25,17710
3,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.4,8.3,140,5500,17,20,23875
4,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,four,108,mpfi,3.50,2.8,8.8,101,5800,23,29,16430


In [4]:
# function for determining a categorical variable vs not (from utils.py)
def iscategorical(x, threshold=0.12):
    """
    determine if x is a categorical variable.


    Inputs:
    ------------------------------------------------------------
    x: pd.DataFrame or np.ndarray, a vector


    Outputs:
    ------------------------------------------------------------
    Bool value
    """
    # convert x to np.ndarray
    if isinstance(x, pd.DataFrame):
        x = x.to_numpy()

    if x.dtype in ["object", "bool", "str"]:
        return True
    elif len(np.unique(x[~np.isnan(x)])) < threshold * len(
        x[~np.isnan(x)]
    ):
        return True
    else:
        return False

In [5]:
# creating a mask for categorical variables the filtering the columns using it
categorical_mask = {col: iscategorical(df[col]) for col in df.columns}
cat_vars = [col for col, val in categorical_mask.items() if val == True]

# making a copy of the original dataframe to keep it unaltered
df_le = df.copy()

for col in cat_vars:
    # label encode the data
    le = LabelEncoder()
    df_le[col] = le.fit_transform(df[col])

df_le.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,4,164,0,1,0,0,3,1,0,99.8,176.6,66.2,54.3,2337,2,2,109,4,3.19,3.4,10.0,102,5500,24,30,13950
1,4,164,0,1,0,0,3,0,0,99.4,176.6,66.4,54.3,2824,2,1,136,4,3.19,3.4,8.0,115,5500,18,22,17450
2,3,158,0,1,0,0,3,1,0,105.8,192.7,71.4,55.7,2844,2,1,136,4,3.19,3.4,8.5,110,5500,19,25,17710
3,3,158,0,1,1,0,3,1,0,105.8,192.7,71.4,55.9,3086,2,1,131,4,3.13,3.4,8.3,140,5500,17,20,23875
4,4,192,1,1,0,1,3,2,0,101.2,176.8,64.8,54.3,2395,2,2,108,4,3.50,2.8,8.8,101,5800,23,29,16430


## Label Encoding

In [6]:
# saving the label encoded dataframe version
df_frac = df_le.copy()
random_state = 20

# set fraction missing in the dataframe
for idx, col in enumerate(df_frac.columns):
    missing = df_frac[col].sample(frac=0.08, random_state=random_state, replace=False).index.to_list()
    df_frac.iloc[missing, idx] = np.nan
    random_state += 2

In [7]:
# separating missing from complete data
missing_idx = {}
complete_idx = {}

for col in df_frac.columns:
    missing_idx[col] = list()
    key = missing_idx[col]
    for idx, rec in enumerate(df_frac[col]):
        if np.isnan(rec):
            key.append(idx)

## One-hot encoding

Impute the data with mean and/or mode for it to work with the ohe encoder.

In [8]:
# # remove target Column
# cat_vars.remove('class')

In [9]:
# dictionary to save the values to be imputed per column
imputed_value = {}

# obtaining the values to use
for col in df_le.columns:
    
    if col in cat_vars:
        try:
            imputed_value[col] = float(df_le[col].mode())
        except TypeError:
            imputed_value[col] = float(df_le[col].mode()[0])

    else:
        imputed_value[col] = float(df_le[col].mean())

In [10]:
# instantiate the ohe encoder
ohe = OneHotEncoder(drop="first", sparse=False)
ohe.fit(df_le[cat_vars])

# one-hot encode the data
cat_transComp = ohe.transform(df_le[cat_vars])
cat_transNames = ohe.get_feature_names_out()

# switch them back to dataframes
cat_oheComp = pd.DataFrame(cat_transComp, columns=cat_transNames, index=df_le.index)
df_oheComp = cat_oheComp.join(df_le[[col for col in df_le.columns if col not in cat_vars]])
df_oheComp.sample(5)

,symboling_1,symboling_2,symboling_3,symboling_4,symboling_5,make_1,make_2,make_3,make_4,make_5,make_6,make_7,make_8,make_9,make_10,make_11,make_12,make_13,make_14,make_15,make_16,make_17,fuel_type_1,aspiration_1,num_of_doors_1,body_style_1,body_style_2,body_style_3,body_style_4,drive_wheels_1,drive_wheels_2,engine_type_1,engine_type_2,engine_type_3,engine_type_4,num_of_cylinders_1,num_of_cylinders_2,num_of_cylinders_3,num_of_cylinders_4,fuel_system_1,fuel_system_2,fuel_system_3,fuel_system_4,fuel_system_5,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
75,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,194,91.3,170.7,67.9,49.7,3139,181,3.43,3.27,7.8,200,5200,17,23,19699
65,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,128,94.5,165.6,63.8,53.3,2028,97,3.15,3.29,9.4,69,5200,31,37,7799
157,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,95,109.1,188.8,68.9,55.5,3217,145,3.01,3.40,23.0,106,4800,26,27,22470
42,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,115,98.8,177.8,66.5,55.5,2425,122,3.39,3.39,8.6,84,4800,26,32,11245
47,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,93,115.6,202.6,71.7,56.3,3770,183,3.58,3.64,21.5,123,4350,22,25,31600


In [11]:
df_oheComp.shape

(159, 59)

In [12]:
# converting previously null records back to null/nan
df_oheMiss = df_oheComp.copy()

for col in missing_idx:
    for missCol in df_oheMiss:
        if col in missCol:
            for idx, rec in enumerate(df_oheMiss[missCol]):
                if idx in missing_idx[col]:
                    df_oheMiss[missCol][idx] = np.nan

## Trying the discriminative DL implementation

In [14]:
num_vars_ = [col for col in df_le.columns if col not in cat_vars]
cat_vars_ = [col for col in df_oheComp.columns if col not in num_vars_]
cat_vars_.append("class")

oneRec = len(df_oheComp.iloc[0, :-1])
oneCol = len(df_oheComp.iloc[:, 1])

Sequential deep learning models require different loss and 'final' activation functions for different types and characteristics of data.

By final activation function I mean the activation function to be used by the layer right before the output layer. 

They are as follows:

- Categorical
    1. Binary Categories
        - Activation = sigmoid
        - Loss = binary crossentropy
    2. Multiple (>2) categories
        - Activation = softmax
        - Loss = categorical crossentropy

- Numerical
    - Activation = ReLu
    - Loss = MSE, RMSE, e.t.c

In the script implementation I will have to think of ways to make sure that this is incorporated within it.

In [15]:
# # instantiate the sequential model
# model = Sequential()

# # add the neccessary input, hidden, dropout and output layers
# model.add(Input(shape=(oneRec, )))
# model.add(Dense(56, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(24, activation='relu'))
# model.add(Dropout(0.05))
# model.add(Dense(4, activation='relu'))
# # model.add(Dense(len(np.unique(df_oheComp.iloc[:, -1])), activation="softmax"))
# model.add(Dense(1))

# model.compile(loss="mse", optimizer='adam', metrics=[metrics.MeanSquaredError()])
# model.fit(df_oheComp.iloc[:, 1:], df_oheComp.iloc[:, 0], epochs=150)
# predictions = model.predict(df_oheComp.iloc[:, 1:])

In [16]:
# predictions

In [17]:
# # investigating the datastructure of the predictions of the deep learning model
# predictions.tolist()[0][0]

In [18]:
# predictions = [round(val)**2 for _ in predictions for val in _]

# for pred, actual in zip(predictions, df_oheComp.iloc[:, -1]):
#     print(pred, actual)

In [19]:
# instantiating the imputer
dl = discriminativeDLImputer(num_vars_, cat_vars_)

fitted_df = dl.fit(df_oheMiss)
# imputed_df = hot.transform(df_oheMiss)

1/1 [==============================] - 0s 159ms/step


In [20]:
print(f"Train data fed to the imputer had an average of {df_oheMiss.isna().sum().mean()} missing values per column.\n")
print(f"The resulting data after imputation has an average of {fitted_df.isna().sum().mean()} missing values per column")

Train data fed to the imputer had an average of 13.0 missing values per column.

The resulting data after imputation has an average of 0.0 missing values per column


In [21]:
fitted_df.sample(8)

,symboling_1,symboling_2,symboling_3,symboling_4,symboling_5,make_1,make_2,make_3,make_4,make_5,make_6,make_7,make_8,make_9,make_10,make_11,make_12,make_13,make_14,make_15,make_16,make_17,fuel_type_1,aspiration_1,num_of_doors_1,body_style_1,body_style_2,body_style_3,body_style_4,drive_wheels_1,drive_wheels_2,engine_type_1,engine_type_2,engine_type_3,engine_type_4,num_of_cylinders_1,num_of_cylinders_2,num_of_cylinders_3,num_of_cylinders_4,fuel_system_1,fuel_system_2,fuel_system_3,fuel_system_4,fuel_system_5,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
11,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,118.0,0.076566,157.300000,63.800000,50.800000,1876.000000,90.0,2.970000,3.230000,9.410000,68.000000,5500.000000,37.000000,41.000000,5572.000000
114,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,95.700000,169.700000,63.600000,59.100000,3110.000000,92.0,3.050000,3.030000,9.000000,62.000000,5169.140137,27.000000,32.000000,8778.000000
132,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,65.0,102.400000,175.600000,66.500000,54.900000,2326.000000,122.0,3.310000,3.540000,8.700000,92.000000,4200.000000,29.000000,34.000000,8948.000000
22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,0.076566,150.000000,64.000000,52.600000,1940.000000,92.0,2.910000,3.410000,9.200000,76.000000,6000.000000,30.000000,34.000000,6529.000000
52,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,161.0,93.000000,157.300000,63.800000,50.800000,2145.000000,98.0,3.030000,3.390000,7.600000,102.000000,5500.000000,24.000000,30.000000,7689.000000
55,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,125.0,96.300000,172.400000,0.097891,51.600000,2365.000000,122.0,3.350000,3.156990,8.500000,88.000000,5000.000000,25.000000,32.000000,6989.000000
29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,85.0,96.500000,175.400000,0.097891,54.100000,2372.000000,110.0,3.150000,3.580000,9.000000,0.181625,4207.064941,27.000000,33.000000,10295.000000
56,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,125.0,96.300000,172.400000,65.400000,51.600000,2405.000000,122.0,3.350000,3.460000,8.500000,88.000000,5000.000000,25.000000,32.000000,10761.582031
115,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,95.700000,166.300000,64.400000,53.000000,2081.000000,98.0,3.190000,3.030000,12.315151,70.000000,4800.000000,0.066211,37.000000,6938.000000
40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,129.0,98.800000,151.668106,66.500000,53.700000,2385.000000,122.0,3.390000,2.008705,8.600000,84.000000,4800.000000,26.000000,-11.521876,10595.000000


Research on activation functions

In [22]:
# fitted_df["class"].value_counts()

In [25]:
for pred, actual in zip(fitted_df["price"], df_oheComp["price"]):
    print(pred, "     ", actual)

13950.0       13950
17450.0       17450
17710.0       17710
23875.0       23875
16430.0       16430
16925.0       16925
20970.0       20970
21105.0       21105
5151.0       5151
6295.0       6295
6575.0       6575
5572.0       5572
6377.0       6377
7957.0       7957
10609.681640625       6229
6692.0       6692
10566.181640625       7609
8921.0       8921
11542.625       12964
6479.0       6479
6855.0       6855
5399.0       5399
6529.0       6529
7129.0       7129
7295.0       7295
7295.0       7295
7895.0       7895
9095.0       9095
8845.0       8845
10295.0       10295
12945.0       12945
10345.0       10345
32250.0       32250
5195.0       5195
6095.0       6095
6795.0       6795
6695.0       6695
7395.0       7395
8845.0       8845
8495.0       8495
10595.0       10595
10245.0       10245
11245.0       11245
18280.0       18280
25552.0       25552
28248.0       28248
11806.9453125       28176
31600.0       31600
10554.244140625       35056
5389.0       5389
6189.0       6189
6669